In [130]:
#####Mixed membership block model#####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import gensim
import itertools
from numpy.random import *
from scipy import optimize
from scipy import sparse
import seaborn as sns

In [131]:
####データの生成####
##データの設定
k = 15   #混合数
d = 3000   #ノード数
hhpt = int(d*(d-1)/2)

In [135]:
##ノード有無を生成
#すべてのノードの組み合わせを作成
d1 = np.repeat(range(d), d).reshape(d, d)
d2 = np.repeat(range(d), d).reshape(d, d, order="F")
id_list1 = [i for i in range(d-1)]
id_list2 = [i for i in range(d-1)]
for i in range(d-1):
    id_list1[i] = d1[i, np.arange(i+1, d)]
    id_list2[i] = d2[i, np.arange(i+1, d)]

#リストをベクトルに変換
d_id01 = np.array(list(itertools.chain(*id_list1)))
d_id02 = np.array(list(itertools.chain(*id_list2)))

#インデックスの設定
d_list = [i for i in range(d)]
for i in range(d):
    d_list[i] = np.array(np.where((d_id01==i) | (d_id02==i))[0], "int")

In [145]:
#ベルヌーイ分布からノードを作成
S = np.ones((hhpt), dtype="int")
for i in range(d):
    prob = np.random.beta(15.0, 35.0, 1)
    S[d_list[i]] = S[d_list[i]] * np.random.binomial(1, prob, d-1)
    
#ノードありの部分のみ抽出
index_s = np.array(np.where(S==1)[0], dtype="int")
d_id1 = d_id01[index_s]
d_id2 = d_id02[index_s]
hhpt = d_id1.shape[0]   #総ノード数

#インデックスの設定
n = np.repeat(0, d)
d_list = [i for i in range(d)]
for i in range(d):
    d_list[i] = np.array(np.where((d_id1==i) | (d_id2==i))[0], "int")
    n[i] = d_list[i].shape[0]

In [146]:
##パラメータを生成
#事前分布を設定
alpha01 = 0.15



0.15